### WikiEngine

In [ ]:
# from wiki.dump import WikiDump
import wiki

path_base = '/Users/harangju/Developer/data/wiki/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

### Exploring the wiki dump

In [ ]:
dump.load_page('Portal:Physics/Topics')
dump.links[:5]

In [ ]:
dump.load_page('Danielle Bassett')
dump.links[:3]

In [ ]:
dump.load_page('Matter', filter_top=True).strip_code()[:200]

### Get index of physics articles

* [all indices on Wikipedia](https://en.wikipedia.org/wiki/Portal:Contents/Indices)
* topics not searched
* international trade ("topics"), theory of constraints (small)
* too big: mathematics, neuroscience

In [ ]:
import string

links = {}
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology', 'chemistry',
          'earth science', 'economics', 'accounting', 'auditing-related',
          'education', 'linguistics', 'law', 'psychology', 'sociology',
          'electronics', 'software engineering', 'robotics']
topics = ['economics']

In [ ]:
for topic in topics:
    dump.load_page('Index of %s articles' % topic)
    links[topic] = [str(l) for l in dump.links]

In [ ]:
links['physics'] = []
for letter in ['!$@', '0–9'] + list(string.ascii_uppercase):
    dump.load_page('Index of physics articles (%s)' % letter)
    links['physics'].extend([str(l) for l in dump.links])

### Build graphs of topics

In [ ]:
import networkx as nx

graphs = {}
for topic in links.keys():
    graphs[topic] = nx.DiGraph()
    wiki.Crawler.bfs(graphs[topic], dump, links[topic], depth_goal=2)
    nx.write_gexf(graphs[topic], path_base + 'graphs/' + topic + '.gexf')

In [ ]:
dump.load_page('Matter')
dump.links